## 1. Installation
Ensure you have Python installed. You can install PyTorch with the following command:
```bash
pip install torch torchvision
```

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the image
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Custom Dataset example
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

## 2. Basics of Tensors

In [ ]:
# Creating Tensors
x = torch.tensor([[1, 2], [3, 4]])
y = torch.rand(2, 2)
z = torch.zeros(2, 2)
w = torch.ones(2, 2)

print(x)
print(y)
print(z)
print(w)

# Tensor Operations
add_result = x + y
mul_result = x * y
dot_result = torch.matmul(x, y)

print(add_result)
print(mul_result)
print(dot_result)

## 3. Data Loading and Preprocessing

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('mnist_data/', download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

data_iter = iter(trainloader)
images, labels = next(data_iter)
print(images.shape)
print(labels.shape)

## 4. Building Neural Networks

In [ ]:
net = SimpleNet()
print(net)

## 5. Loss Function and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

## 6. Training the Network

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()  # zero the parameter gradients

        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss/len(trainloader)}')

## 7. Evaluation

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in trainloader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

## 8. Saving and Loading Models

In [ ]:
torch.save(net.state_dict(), 'simple_net.pth')
net.load_state_dict(torch.load('simple_net.pth'))
net.eval()

## 9. Moving to GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = x.to(device)
net.to(device)

## 10. Advanced Topics

### Transfer Learning

In [ ]:
# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Freeze the feature extraction layers
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer to match the number of classes in the new dataset
model.fc = nn.Linear(model.fc.in_features, 10)

# Move the model to GPU
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)

# Train only the final layer
for epoch in range(5):
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss/len(trainloader)}')

## Custom Data Reading

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image

# Example of custom data reading
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name)
        y_label = torch.tensor(int(self.annotations.iloc[idx, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

# Usage example
custom_dataset = CustomImageDataset(csv_file='data/labels.csv', root_dir='data/images', transform=transform)
custom_loader = DataLoader(custom_dataset, batch_size=64, shuffle=True)

## Scheduler

In [ ]:
# Using a learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Training loop with scheduler step
for epoch in range(5):
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    scheduler.step()  # Step the learning rate scheduler
    print(f'Epoch {epoch + 1}, Loss: {running_loss/len(trainloader)}, Learning Rate: {scheduler.get_last_lr()[0]}')

## Data Augmentation

In [ ]:
# Data augmentation with torchvision transforms
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Apply data augmentation in the dataset
augmented_dataset = datasets.ImageFolder(root='data/train', transform=data_transforms)
augmented_loader = DataLoader(augmented_dataset, batch_size=64, shuffle=True)

## Custom Data Augmentation

In [ ]:
# Custom data augmentation
class CustomAugmentation:
    def __call__(self, image):
        # Example: Convert image to grayscale and then back to RGB
        image = transforms.functional.to_grayscale(image, num_output_channels=3)
        image = transforms.functional.adjust_contrast(image, 2)
        return image

custom_transforms = transforms.Compose([
    CustomAugmentation(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# Apply custom data augmentation in the dataset
custom_augmented_dataset = CustomImageDataset(csv_file='data/labels.csv', root_dir='data/images', transform=custom_transforms)
custom_augmented_loader = DataLoader(custom_augmented_dataset, batch_size=64, shuffle=True)